In [3]:
from keras.datasets import imdb
(x_train, y_train), (x_test, y_test) = imdb.load_data()
cutoff = 500

In [4]:
from keras.preprocessing import sequence
x_train_padded = sequence.pad_sequences(x_train, maxlen=cutoff)
x_test_padded = sequence.pad_sequences(x_test, maxlen=cutoff)

In [5]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense

In [6]:
lstm_model = Sequential()
lstm_model.add(Embedding(input_dim=len(imdb.get_word_index())+3, output_dim=100, input_length=cutoff))
# return_sequences tells the LSTM to output the full sequence, for use by the next LSTM layer. The final
# LSTM layer should return only the output sequence, for use in the Dense output layer
lstm_model.add(LSTM(units=32, return_sequences=True))
lstm_model.add(LSTM(units=32))
lstm_model.add(Dense(units=1, activation='sigmoid')) # because at the end, we want one yes/no answer
lstm_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])

In [7]:
lstm_model.fit(x_train_padded, y_train, epochs=1, batch_size=64)

Epoch 1/1
25000/25000 [==============================] - 716s 29ms/step - loss: 0.3986 - binary_accuracy: 0.8164


In [8]:
lstm_scores = lstm_model.evaluate(x_test_padded, y_test)
print('loss: {} accuracy: {}'.format(*lstm_scores))

25000/25000 [==============================] - 76s 3ms/step
loss: 0.2983583768177033 accuracy: 0.87684


In [27]:
from keras.layers import GRU, SimpleRNN
new_model = Sequential()
new_model.add(Embedding(input_dim=len(imdb.get_word_index())+3, output_dim=100, input_length=cutoff))
new_model.add(LSTM(units=32, return_sequences=True))
new_model.add(GRU(units=32, return_sequences=True))
new_model.add(SimpleRNN(units=32, return_sequences=True))
new_model.add(LSTM(units=32))
new_model.add(Dense(units=1, activation='sigmoid'))
new_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])

In [28]:
new_model.fit(x_train_padded, y_train, epochs=1, batch_size=64)

Epoch 1/1
25000/25000 [==============================] - 420s 17ms/step - loss: 0.4255 - binary_accuracy: 0.7921


In [29]:
new_scores = new_model.evaluate(x_test_padded, y_test)
print('loss: {} accuracy: {}'.format(*new_scores))

25000/25000 [==============================] - 135s 5ms/step
loss: 0.3410003401470184 accuracy: 0.85564


In [23]:
import numpy

In [30]:
one_unit_SRNN = Sequential()
one_unit_SRNN.add(SimpleRNN(units=1, input_shape=(None, 1), activation='linear', use_bias=False))

In [31]:
one_unit_SRNN_weights = one_unit_SRNN.get_weights()
one_unit_SRNN_weights

[array([[0.5857984]], dtype=float32), array([[-1.]], dtype=float32)]

In [32]:
one_unit_SRNN_weights[0][0][0] = 1
one_unit_SRNN_weights[1][0][0] = 1
one_unit_SRNN.set_weights(one_unit_SRNN_weights)
one_unit_SRNN.get_weights()

[array([[1.]], dtype=float32), array([[1.]], dtype=float32)]

In [33]:
one_unit_SRNN.predict(numpy.array([ [[3], [3], [7]] ]))

array([[13.]], dtype=float32)

In [50]:
one_unit_SRNN.predict(numpy.array([ [[4], [4], [8]] ]))

array([[16.]], dtype=float32)

In [83]:
one_unit_SRNN_weights[0][0][0] = 2
one_unit_SRNN_weights[1][0][0] = 1
one_unit_SRNN.set_weights(one_unit_SRNN_weights)
one_unit_SRNN.get_weights()
one_unit_SRNN.predict(numpy.array([ [[4], [4], [8]] ])) #all by 2

array([[32.]], dtype=float32)

In [97]:
one_unit_SRNN_weights[0][0][0] = 1
one_unit_SRNN_weights[1][0][0] = 2
one_unit_SRNN.set_weights(one_unit_SRNN_weights)
one_unit_SRNN.get_weights()
one_unit_SRNN.predict(numpy.array([ [[4], [4], [8]] ])) #by 4, 2, 1

array([[32.]], dtype=float32)

In [104]:
one_unit_SRNN_weights[0][0][0] = 2
one_unit_SRNN_weights[1][0][0] = 2
one_unit_SRNN.set_weights(one_unit_SRNN_weights)
one_unit_SRNN.get_weights()
one_unit_SRNN.predict(numpy.array([ [[4], [4], [8]] ])) #by 8, 4, 2

array([[64.]], dtype=float32)

In [111]:
one_unit_SRNN_weights[0][0][0] = 1
one_unit_SRNN_weights[1][0][0] = 3
one_unit_SRNN.set_weights(one_unit_SRNN_weights)
one_unit_SRNN.get_weights()
one_unit_SRNN.predict(numpy.array([ [[4], [4], [8]] ])) #by 9, 3, 1

array([[56.]], dtype=float32)

In [120]:
one_unit_SRNN_weights[0][0][0] = 1
one_unit_SRNN_weights[1][0][0] = -1
one_unit_SRNN.set_weights(one_unit_SRNN_weights)
one_unit_SRNN.get_weights()
one_unit_SRNN.predict(numpy.array([ [[4], [4], [8]] ])) #by 1, -1, 1

array([[8.]], dtype=float32)

In [126]:
one_unit_SRNN_weights[0][0][0] = 1
one_unit_SRNN_weights[1][0][0] = 0
one_unit_SRNN.set_weights(one_unit_SRNN_weights)
one_unit_SRNN.get_weights()
one_unit_SRNN.predict(numpy.array([ [[31546], [1243], [8]] ])) #by 0, 0, 1

array([[8.]], dtype=float32)

The values (a, b, and c) and weights (w1 and w2) seem to act in the relationship w1(a(w2^2)+b(w2)+c)

In [127]:
two_unit_SRNN = Sequential()
two_unit_SRNN.add(SimpleRNN(units=2, input_shape=(None, 1), activation='linear', use_bias=False))

In [128]:
two_unit_SRNN_weights = two_unit_SRNN.get_weights()
two_unit_SRNN_weights

[array([[-1.1472132, -0.1190449]], dtype=float32),
 array([[ 0.07826269, -0.99693274],
        [-0.99693274, -0.07826269]], dtype=float32)]

In [471]:
two_unit_SRNN_weights[0][0][0] = 1
two_unit_SRNN_weights[0][0][1] = 1
two_unit_SRNN_weights[1][0][0] = 1
two_unit_SRNN_weights[1][0][1] = 0
two_unit_SRNN_weights[1][1][0] = 0
two_unit_SRNN_weights[1][1][1] = 0
two_unit_SRNN.set_weights(two_unit_SRNN_weights)
two_unit_SRNN.get_weights()

[array([[1., 1.]], dtype=float32), array([[1., 0.],
        [0., 0.]], dtype=float32)]

In [472]:
two_unit_SRNN.predict(numpy.array([ [[3], [3], [7], [5]] ]))

array([[18.,  5.]], dtype=float32)

In [480]:
two_unit_SRNN_weights[0][0][0] = 1 #w1
w1 = two_unit_SRNN_weights[0][0][0]
two_unit_SRNN_weights[0][0][1] = 1 #w2
w2 = two_unit_SRNN_weights[0][0][1]
two_unit_SRNN_weights[1][0][0] = 1 #w3
w3 = two_unit_SRNN_weights[1][0][0]
two_unit_SRNN_weights[1][0][1] = 0 #w4
w4 = two_unit_SRNN_weights[1][0][1]
two_unit_SRNN_weights[1][1][0] = 0 #w5
w5 = two_unit_SRNN_weights[1][1][0]
two_unit_SRNN_weights[1][1][1] = 2 #w6 EXPONENTIAL THING?
w6 = two_unit_SRNN_weights[1][1][1]
two_unit_SRNN.set_weights(two_unit_SRNN_weights)
a=3
b=3
c=7
d=5
print(w1*(a*(w3**3)+b*(w3**2)+c*(w3)+d), (w1*w4*a*(w3**2)+w1*w4*b*(w3)+w1*w4*c+w2*d))
#print(a*(w3**2),b*(w3),c)
two_unit_SRNN.predict(numpy.array([ [[a], [b], [c], [d]] ])) #a,b,c,d

18.0 5.0


array([[18., 55.]], dtype=float32)

w1(a(w3^3)+b(w3^2)+c(w3)+d), w2(a(w3*w4^3)+b(w3*w4^2)+c(w3*w4)+d)